In [2]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 35.2 MB/s eta 0:00:00


In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model

# Step 1: Download IBM price data and compute log returns
IBM_dprice = yf.download('IBM', start="2020-01-01", end="2022-01-01", auto_adjust=True)
IBM_dprice['log_return'] = 100*np.log(IBM_dprice['Close'] / IBM_dprice['Close'].shift(1))
IBM_log_return = IBM_dprice['log_return'].dropna()

# Step 2: Fit an ARMA(0,0) model (i.e., a constant mean model) to the log returns
arma_model = ARIMA(IBM_log_return, order=(0, 0, 0))
arma_result = arma_model.fit()

# Forecast the next period mean return
# Instead of using [0], use .iloc[0] to access the first element regardless of the index label.
forecast_series = arma_result.forecast(steps=1)
mean_forecast = forecast_series.iloc[0]
print("Forecasted mean return:", mean_forecast)

# Step 3: Estimate a GARCH(1,1) model on the ARMA residuals
residuals = arma_result.resid
garch_model = arch_model(residuals, vol='Garch', p=1, q=1, mean='Zero', dist='normal')
garch_result = garch_model.fit(disp='off')

# Forecast the next period volatility (sigma)
garch_forecast = garch_result.forecast(horizon=1)

# It is often useful to inspect the forecast variance DataFrame
print("Forecast variance DataFrame:")
print(garch_forecast.variance)

# The forecast variance is stored in a DataFrame where each row corresponds to a date
# and each column corresponds to the forecast horizon. For horizon=1, extract the first forecast value.
# We use .iloc[-1] to select the last row (most recent forecast) and then .values[0] to get the first element.
sigma_forecast = np.sqrt(garch_forecast.variance.iloc[-1].values[0])
print("Forecasted sigma:", sigma_forecast)

# Step 4: Compute the VaR for the loss L_{t+1} = - (r_{t+1} + 1.65 * sigma_{t+1})
VaR = -mean_forecast + 1.65 * sigma_forecast
print("The VaR at 95% confidence level is:", VaR)


[*********************100%***********************]  1 of 1 completed

Forecasted mean return: 0.02620431905036155
Forecast variance DataFrame:
                 h.1
Date                
2021-12-31  1.902936
Forecasted sigma: 1.3794695447653558
The VaR at 95% confidence level is: 2.2499204298124753



/home/carlo/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/carlo/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/carlo/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/carlo/miniconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`